In [1]:
%run C:/Users/admin/Desktop/EeD/Python/Project/Preprocessing.ipynb


[8.50072216e-03 1.74424579e-02 1.90258166e-02 1.41926221e-02
 1.05307382e-02 1.75941207e-02 1.86468741e-02 1.36650152e-02
 1.73423863e-02 4.58490174e-03 1.96355884e-02 1.81671827e-02
 1.97782799e-02 1.18805601e-02 1.58888453e-03 1.83766878e-02
 9.83643636e-03 4.03676992e-03 6.08887102e-03 7.24964478e-03
 9.72363951e-03 1.84590394e-03 1.58301781e-02 9.92865958e-03
 1.56500173e-02 1.08685400e-02 1.88741872e-02 1.64065999e-02
 1.35702470e-02 6.16415391e-03 2.34636559e-03 6.27184452e-03
 6.14683426e-04 2.66823220e-03 2.07657186e-02 1.81057414e-02
 5.58647754e-04 1.66390814e-03 7.73375455e-04 4.60974544e-03
 3.27201705e-03 2.88926224e-04 4.90840667e-04 1.74881828e-03
 1.52359560e-03 7.41638106e-03 2.49423555e-03 9.57096837e-04
 7.08313241e-03 1.62338611e-03 1.53961866e-03 1.48326783e-03
 3.64729896e-03 2.22162779e-04 1.14331351e-04 7.00984111e-03
 3.89130124e-03 3.18057722e-03 6.56659233e-04 7.45040078e-03
 1.56988241e-03 2.31488239e-03 1.18028217e-03 9.30657331e-04
 3.29582514e-05 3.771224